In [1]:
from twisted.internet import defer
import time

In [3]:
d = defer.Deferred()

In [4]:
d.called

False

In [5]:
d.callback(3)

In [6]:
d.called

True

In [7]:
d.result

3

In [2]:
def install_wordpress(customer):
    print('starting install for {}'.format(customer))
    time.sleep(3)
    print('all done for {}'.format(customer))

In [19]:
@multi_args
def developer_day(customers):
    for customer in customers:
        install_wordpress(customer)

In [24]:
customers = ['custom_%02d' %i for i in range(15)]

In [5]:
customers

['custom_00', 'custom_01', 'custom_02', 'custom_03', 'custom_04']

In [17]:
def multi_args(func):
    def wrapper(*args,**kwargs):
        start=time.time()
        func(*args,**kwargs)
        print('time used: {}'.format(time.time()-start))
    return wrapper

In [20]:
developer_day(customers)

starting install for custom_00
all done for custom_00
starting install for custom_01
all done for custom_01
starting install for custom_02
all done for custom_02
starting install for custom_03
all done for custom_03
starting install for custom_04
all done for custom_04
time used: 15.000999927520752


In [22]:
import threading
@multi_args
def developers_day_thread(customers):
    lock = threading.Lock()
    
    def dev_day(id):
        lock.acquire()
        while customers:
            customer = customers.pop(0)
            lock.release()
            install_wordpress(customer)
            lock.acquire()
        lock.release()
        print('Bye from developer {}'.format(id))
    
    thread = [threading.Thread(target=dev_day,args=(i,)) for i in range(5)]
    for i in thread:
        i.start()
    for i in thread:
        i.join()
        

In [25]:
developers_day_thread(customers)

starting install for custom_00
starting install for custom_01
starting install for custom_02
starting install for custom_03
starting install for custom_04
all done for custom_00
starting install for custom_05
all done for custom_02all done for custom_01
starting install for custom_06

starting install for custom_07
all done for custom_03
starting install for custom_08
all done for custom_04
starting install for custom_09
all done for custom_05
starting install for custom_10
all done for custom_07all done for custom_06
starting install for custom_11

starting install for custom_12
all done for custom_08
starting install for custom_13
all done for custom_09
starting install for custom_14
all done for custom_10
Bye from developer 0
all done for custom_11all done for custom_12
Bye from developer 1

Bye from developer 2
all done for custom_13
Bye from developer 3
all done for custom_14
Bye from developer 4
time used: 9.04800009727478


In [1]:
from twisted.internet import reactor,defer,task

In [2]:
def schedule_install(customer):
    
    def schedule_install_wordpress():
        
        def on_done():
            print('Callback: finished installation for {}'.format(customer))
        
    
        print('Scheduling: installation for {}'.format(customer))
        return task.deferLater(reactor,3,on_done)
    
    
    
    def all_done(_):
        print('all done for {}'.format(customer))
        
    d=schedule_install_wordpress()
    d.addCallback(all_done)
    return d

In [3]:
def twisted_develop_day(customers):
    print('Goodmorning from twisted developer')
    work = [schedule_install(customer) for customer in customers]
    join = defer.DeferredList(work)
    join.addCallback(lambda _: reactor.stop())
    print('Bye from twisterd developers')

In [4]:
twisted_develop_day(["customer %02d" %i for i in range(15)])

Goodmorning from twisted developer
Scheduling: installation for customer 00
Scheduling: installation for customer 01
Scheduling: installation for customer 02
Scheduling: installation for customer 03
Scheduling: installation for customer 04
Scheduling: installation for customer 05
Scheduling: installation for customer 06
Scheduling: installation for customer 07
Scheduling: installation for customer 08
Scheduling: installation for customer 09
Scheduling: installation for customer 10
Scheduling: installation for customer 11
Scheduling: installation for customer 12
Scheduling: installation for customer 13
Scheduling: installation for customer 14
Bye from twisterd developers


In [5]:
# start=time.time()
reactor.run()
# print('time used {}'.format(time.time()-start))

Callback: finished installation for customer 00
Callback: finished installation for customer 02
Callback: finished installation for customer 06
Callback: finished installation for customer 05
Callback: finished installation for customer 01
Callback: finished installation for customer 04
Callback: finished installation for customer 09
Callback: finished installation for customer 03
Callback: finished installation for customer 08
Callback: finished installation for customer 07
Callback: finished installation for customer 13
Callback: finished installation for customer 12
Callback: finished installation for customer 14
Callback: finished installation for customer 11
Callback: finished installation for customer 10
